In [1]:
from telethon.sync import TelegramClient
from client import  parse_channel_messages
from config import API_ID, API_HASH, TOKEN
import pandas as pd

import torch
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizerFast

/home/daniil/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
client = TelegramClient('session_name', API_ID, API_HASH)

In [3]:
ans = await parse_channel_messages(client, 'tinkoff_analytics_official', limit=200)

In [4]:
df = pd.DataFrame({"text": ans})
df

,text
0,**Мосбиржа: повышаем таргет**\nНедавно биржа о...
1,[​​](https://telegra.ph/file/84709e42b65175098...
2,[​​](https://telegra.ph/file/2f70d2ef61e576c93...
3,__Как торговали участники российского рынка ак...
4,[​​](https://telegra.ph/file/3274e62c7924893fd...
...,...
195,**Лукойл: повышаем таргет **\nСегодня акции Лу...
196,В 3-м квартале объемы производства стали у ММК...
197,**Positive Technologies: закрываем идею на пок...
198,


In [5]:
df = df[df.text != '']
df

,text
0,**Мосбиржа: повышаем таргет**\nНедавно биржа о...
1,[​​](https://telegra.ph/file/84709e42b65175098...
2,[​​](https://telegra.ph/file/2f70d2ef61e576c93...
3,__Как торговали участники российского рынка ак...
4,[​​](https://telegra.ph/file/3274e62c7924893fd...
...,...
193,[​​](https://telegra.ph/file/c259915c57024641a...
194,**Сырьевые рынки: на кого делать ставку?\n**В ...
195,**Лукойл: повышаем таргет **\nСегодня акции Лу...
196,В 3-м квартале объемы производства стали у ММК...


In [6]:
df.to_csv("dataset_tinkoff_analytics.csv")

In [11]:
df = pd.read_csv('dataset2_labled.csv', index_col=0)[['text', 'sentiment']].reset_index(drop=True)
df

,text,sentiment
0,#AFKS #Отчетность\n🖨 АФК Система: МСФО 2023\n\...,Positive
1,#ENPG #Идея\n🧠 **Мозговой центр Сигналов РЦБ з...,Neutral
2,#MGNT\n**Магнит в моменте достиг 8000 рублей —...,Positive
3,#Политика\n🇷🇺 Запрет взыскания и ареста денег/...,Neutral
4,#GLTR\n⚡️ **Британский инвестфонд Southey Capi...,Neutral
...,...,...
143,#GMKN\n**📊**** Динамика корзины металлов Норни...,Negative
144,#NVTK\n🇷🇺 План грузооборота по Севморпути на 2...,Positive
145,#Нефтянка \n🇷🇺 РФ снижает добычу нефти во 2кв2...,Neutral
146,#LKOH\n🛢 Лукойл может в течение 1-2 месяцев во...,Positive


In [15]:
ans = df.text.to_list()

In [13]:
tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment-rusentiment')
model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment-rusentiment', return_dict=True)


In [22]:
for i in range(1):
    text = df.text[i]
    label = df.sentiment[i]
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted = torch.argmax(predicted, dim=1).numpy()
    print('Predict', predicted[0])
    print('True', label)
    print(inputs)
    
    print()

Predict 0
True Positive
{'input_ids': tensor([[  101,   110, 18731,   191,   207,   110, 71427,  2083,   100, 89444,
           452, 22230,   156, 29202, 36344, 89459, 65588,  1816,   138,   132,
         11326, 45102,  2007,   120,   126,  2946,   114,   368,   134,   368,
           122, 67336, 40070,   146,   132,   142,  7246,  2007,   120, 16353,
          6612, 13074, 10320,   132,   146,  7246,   122,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

